# Two events(possibly  separated in space and Time)
derives heavily from the code: https://github.com/AnmolHarshana/GLEE/blob/411e735f12c925a0681716334e32f86b5675f65e/Seismology_Simulations/Iteration_receiver_source.ipynb

In [1]:
# importing the requiered libraries

import numpy as np
import matplotlib.pyplot as plt
from sympy import *


In [2]:
### Testing out with 101 receivers placed in a line
n_rec=11# number of receivers, to be kept odd fro first run

rec_pos_act= np.zeros(shape =(n_rec,2))  # Assigning thr actual receiver position

v = 200 # m/s ### velocity of propagation (in sand)
freq=2 #Hz, max recorded frequency
lamda=v/freq# wavelength
d=2000#m,  l=2000m  > 10 times lamda 
src=(d,3*d)


for i in range(0,n_rec):
    rec_pos_act[i,1]=0
    rec_pos_act[i,0]=(i-(n_rec-1)/2)*5 #first receievr at (-50,0) last at (50,0), all in line, distance 5 between them 
        
###rec_pos_act[:,:] = 0# The deployer is at origin. 
#noise_r = np.random.normal(loc=0.0,scale =100,size = (101,2))
t_rec_ideal = np.zeros(shape =(n_rec,))  # time recorded in each receiver; form: (index of receiver,time) 

rec_pos_mes= rec_pos_act #+noise_r  # rec_pos_mes stores[(x1,y1),(x2,y2),..]

for i in range (0,n_rec):
    t_rec_ideal[0:n_rec] = np.sqrt((rec_pos_mes[0:n_rec,0]-src[0])**2 + (rec_pos_mes[0:n_rec,1]-src[1])**2)/v


t_Rec_real=t_rec_ideal
### t_Rec_real has form [[rec1 recording], [rec2 recording]]
t_diff=np.zeros(shape=(n_rec,))
for i in range(0,n_rec):
    t_diff[i]=t_rec_ideal[i]-t_rec_ideal[0]


In [3]:
def sum_of_errors(s_x,s_y,n_rec,rec_loc,time_diff):
    soe=0
    for i in range (0,n_rec):
        soe+=(time_diff[i]-s_x*rec_loc[i,0]-s_y*rec_loc[i,1])**2
    return soe    

In [4]:
def estimate_slowness(v,s_x_min,s_y_min,s_x_max,s_y_max,s_x_step,s_y_step,n_rec,rec_loc,time_diff):
    s_x_estimate=0
    s_y_estimate=0
    error=10000
    s_x_range = np.linspace(s_x_min,s_x_max,s_x_step)
    for s_x_temp in s_x_range :
        s_y_range = np.linspace(s_y_min,s_y_max,s_y_step)
        for s_y_temp in s_y_range:
            soe=sum_of_errors(s_x_temp,s_y_temp,n_rec,rec_loc,time_diff)  
            #print(s_x_temp,s_y_temp)
            #print(soe)
            #print(s_x_temp,s_y_temp)
            if ( error>soe):
                error =soe
                s_x_estimate=s_x_temp
                s_y_estimate=s_y_temp
                
    return (s_x_estimate,s_y_estimate,error)    

In [5]:
s_x_estimated,s_y_estimated,final_error=estimate_slowness(v,-1/v,-1/v,1/v,1/v,100,100,n_rec,rec_pos_mes,t_diff)
print("s_x_estimated: " ,s_x_estimated,"s_y_estimated :",s_y_estimated,"final_error: ", final_error)

s_x_estimated:  -0.0015656565656565658 s_y_estimated : -0.005 final_error:  0.017304194989893842


In [6]:
sinphi=-v*s_x_estimated
cosphi=-v*s_y_estimated
print(np.arcsin(sinphi),np.arccos(cosphi))

0.3184883743170764 0.0


In [8]:
print("actual angle: ", np.arctan(src[1]/src[0]))

actual angle:  1.2490457723982544
